# Project Plan

- Write a web scraper that scrapes title, description, metrics etc. on a list of marvel superheroes from Wikipedia
- Uses PyMYSQL to write to a database 
	- Several tables 
- Data Modeling in either PowerBI or SQL itself
- Visualization that looks very professional and fun


Table 1 - Character Table
ID | Name | CharacterDesignation | Description | CharacterLink | NumAppearances 

Table 2 - Actor
ID | Actor | Character | Age | NumFilms | YearsInMCU

Table 3 - Films
ID | Title | PremiereDate | Director | Producer | Starring | Cinematography | ProductionCompany| Distributor | Country | Language | Screenplay

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import csv
import pymysql

# Table 1: Scraper

In [2]:
# https://en.wikipedia.org/wiki/Characters_of_the_Marvel_Cinematic_Universe

'''class Character(self, name, designation, description, link):
    self.name = name
    self.designation = designation
    self.description = description
    self.link = link
'''
def get_major_characters():
    html = requests.get('https://en.wikipedia.org/wiki/Characters_of_the_Marvel_Cinematic_Universe')
    bs = BeautifulSoup(html.text, 'lxml')
    
    #Character Names
    try:
        names = bs.find_all('h3')
        names = [names[i].getText().rstrip('[edit]') for i in range(len(names)-2) if not names[i].getText().startswith('Introduced')]
    except:
        print('There was an error pulling character names.')
    #Character Designation
    designation = 'Major Character'
    
    #Character Descriptions
    descriptions = []
    try:
        for heading in bs.find_all('h3'):
            descriptions.append(heading.findNext('p').getText())
    except:
        print('There was an error in pulling descriptions.')
    
    #Movie Appearances
    appearances = []
    try: 
        for heading in bs.find_all('h3'):
            appearances.append(heading.find_previous_sibling('p').getText())
    except:
        print('There was an error in pulling appearances.')
    
    #Character Link
    links = []
    try:
        for div in bs.find_all('div',{'role':'note'}):
            links.append(div.find('a',href = re.compile('^(/wiki/)')))
    except:
        print('There was an error in getting links')
                
    return links


In [3]:
appearances = get_major_characters()

There was an error in pulling descriptions.
There was an error in pulling appearances.
